This notebook will be converted into a distributable python script. The script will exist in a distributable file package with the following architecture:

```
Strain_modeler/  
|-- project1/  
    |-- geometry/  
    |-- dummies/
    |-- output/  
|-- scripts/
|-- README.md
```

1. The user will create a project folder, add a .xyz geometry of the optimized strained molecule, and as many .out gaussian output files from force calculations as necessary into the input folder.
2. A script will compile and average all of the .out files to create .tcl scripts in the output folder.
3. The user can then load the .xyz geometry into VMD and run the .tcl scripts to visualize each type of molecular strain.

In [1]:
from scripts import map_forces, vmd_norm, vmd_writer, force_parse
import os

In [2]:
for file in os.listdir('geometry'):
    if file.endswith(".xyz"):
        geometry_filename = 'geometry/' + file

In [3]:
full_bond_forces = []
full_angle_forces = []
full_dihedral_forces = []
full_key = []
for file in os.listdir('dummies'):
    if file.endswith(".out"):
        a, b, c = map_forces(geometry_filename, file)
        for line in a:
            full_bond_forces.append(line)
        for line in b:
            full_angle_forces.append(line)
        for line in c:
            full_dihedral_forces.append(line)

In [4]:
full_bond_list = []
for line in full_bond_forces:
    full_bond_list.append(line[1])

In [5]:
bond_list = []
for line in full_bond_list:
    if line in bond_list:
        continue
    else:
        bond_list.append(line)

In [6]:
bond_forces = []
for line in bond_list:
    bond_forces.append([line])

In [7]:
for line in full_bond_forces:
    for bond in bond_forces:
        if bond[0] == line[1]:
            bond.append(line[0])

In [8]:
bond_force = []
for line in bond_forces:
    a = 0
    for value in line[1:]:
        a += value
    b = a/(len(line)-1)
    bond_force.append([b, line[0]])

In [9]:
bond_force_vmd = vmd_norm(bond_force)

In [10]:
vmd_writer("vmd_bond_script_total.tcl", bond_force_vmd, geometry_filename)